In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
words = open('names.txt', 'r').read().splitlines()

In [3]:
# define stoi 
stoi = {}
allletters = sorted(set("".join(words)))

stoi = {s:i+1 for i,s in enumerate(allletters)}
stoi['.'] = 0

itos = {i:s for s,i in stoi.items()}

In [10]:
X , Y = [], []
block_size= 3 # can be reset to whatever you like

for w in words[:5]:
    #'emma'
    print(w)
    context = [0]*block_size # contains indcies of context letters
    for ch in w + '.':
        ix = stoi[ch] # 'e' -> 5
        Y.append(ix) # 5 is the target
        X.append(context)
        print("".join(itos[i] for i in context), '------->', ch)
        context = context[1:] + [ix] # update context and append new index

X = torch.tensor(X)
Y = torch.tensor(Y)


emma
... -------> e
..e -------> m
.em -------> m
emm -------> a
mma -------> .
olivia
... -------> o
..o -------> l
.ol -------> i
oli -------> v
liv -------> i
ivi -------> a
via -------> .
ava
... -------> a
..a -------> v
.av -------> a
ava -------> .
isabella
... -------> i
..i -------> s
.is -------> a
isa -------> b
sab -------> e
abe -------> l
bel -------> l
ell -------> a
lla -------> .
sophia
... -------> s
..s -------> o
.so -------> p
sop -------> h
oph -------> i
phi -------> a
hia -------> .


In [11]:
g = torch.Generator().manual_seed(214748367)
# lookup matrix
C = torch.randn((27,2), generator=g)
# hidden layer - 100 neurons
W1 = torch.randn((6,100), generator=g)
b1 = torch.randn((100,), generator=g)
# Output layer
W2 = torch.randn((100,27), generator=g )
b2 = torch.randn((27,), generator=g)

parameters = [C, W1, b1, W2, b2]

In [12]:
sum(p.nelement() for p in parameters) # total no of parameters in the network

3481

I have combined many steps from previous notebook, so pl refer that in case of confusion. 

`X.shape = (32,3)`; 32 is no of datapoints in first 5 words. 

In [15]:
emb = C[X]
H = torch.tanh(emb.view(emb.shape[0], -1) @ W1 + b1) # 32,100
probs = torch.softmax(H @ W2 + b2, dim = 1) # 32,27
loss = -probs[torch.arange(32), Y].log().mean()
loss

tensor(20.6727)